In [7]:
import sys
sys.path.insert(1, '../../Src')
from utils.preprocessing import *
import training

import numpy as np
import pandas as pd
import os


from math import ceil
import warnings


warnings.filterwarnings('ignore')

PATH_DATASET = '../dataset/'
PATH_RESULTS = '../../results/Demanda/'


FILL = False
TARGET = 'Demanda'


N_FOLDS = 3
N_FEATURE_IMPORTANCE = 15

plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
SEED = 42
seed_everything(SEED)


%load_ext autoreload
%autoreload 2
warnings.simplefilter(action='ignore', category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Lectura de data referencial para sacar metricas de analisis

In [2]:
#submission_top_reference = 'Submission_73.csv'
submission_top_reference = 'submission_weeks3_custom_pred_0.5_pred_0.5_real_rmse_kaggle_0.247.csv'
print('reading .. ',os.path.join('../../results/',submission_top_reference))
result = pd.read_csv(os.path.join('../../results/',submission_top_reference))
descriptive_columns = ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']
reverse_mapping_file = '../../utils/reverse_dict_mapping_list.txt'

result[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']] = result['ID'].str.split('|',expand=True)

with open(reverse_mapping_file, 'rb') as f:
    reverse_mapping = pickle.load( f)
    
descriptive_columns = ['Z_MARCA', 'Z_GAMA', 'Z_MODELO',
                       'Z_DEPARTAMENTO', 'Z_PUNTO_VENTA']
i=0
for column in descriptive_columns:
    if column in ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']:
        inv_reverse_mapping = {v: k for k, v in reverse_mapping[i].items()}

        result[column] = result[column].map(inv_reverse_mapping)
    i+=1
    
result = result.rename(columns = {TARGET: TARGET+'_real'})
result.head(1)

reading ..  ../../results/submission_weeks3_custom_pred_0.5_pred_0.5_real_rmse_kaggle_0.247.csv


,ID,Demanda_real,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK
0,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.010752,MOD_170,PVENT_259,GAM_2,SEMANA_51


# Lectura de la data con las nuevas variables creadas

In [3]:
print('Reading dataset')
SHIFT=1
df_train  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','df_train_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))
df_test   = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','df_test_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))

gc.collect()
print('df_train size :',df_train.shape)
print('df_test size  :',df_test.shape) 

features_names = set(df_train.columns)-set([TARGET,'date_block_num',TARGET+'_clipped','release','item_id',
                                            'Z_WEEK_DATE','date_block_num','year',
                                            'month', 'is_month_end', 'tm_m_end','is_month_start','tm_wm', 'tm_w_end','is_quarter_start','is_year_start',
                                             'day_of_week',
                                            'day_of_year', 'day',
                                           ]) 
print(len(features_names))
features_names = list(features_names)

df_train['week_of_month'] = df_train['week_of_month'].astype('category')
df_test['week_of_month']  = df_test['week_of_month'].astype('category')
total_features_names = features_names.copy()

Reading dataset
df_train size : (2358650, 1111)
df_test size  : (471730, 1111)
1094


# Filtro de datos por ventana de tiempo temporal [1 a 3 meses antes]

In [4]:
def get_feature_names_shift(shift,total_features_names):
    features_names = []

    for column in total_features_names:
        if 'shift' in column:
            number = column.split('shift_')[1]
            number = int(number.split('_')[0])
            if number>=shift and number<=(shift+2):
                features_names.append(column)
        else:
            features_names.append(column)
    return features_names

# Definir la ventana de testing para poder evaluar que el modelo predice bien el futuro
### esta ventana de testing es nuestra base de analisis en el training

In [5]:
TRAIN_START = '2021-08-01'
TEST_START = '2022-03-01'
TEST_END   = '2022-05-01'
print(df_train.shape)
df_train = df_train[(df_train['Z_WEEK_DATE'] >= TRAIN_START)]
print(df_train.shape)

(2358650, 1111)
(1839747, 1111)


# Entrenamiento con cross validation e importancia de datos

In [9]:
PATH_RESULTS = '../../results/Demanda/'
training.PATH_RESULTS = PATH_RESULTS
training.submission_analysis.TARGET  = TARGET

metrics_dict_list = []
for SHIFT in range(1,5):
    print('*'*20)
    print('*'*20)
    print('WEEK = ',SHIFT)
    print('*'*20)
    print('*'*20)

    features_names = get_feature_names_shift(SHIFT,total_features_names)
    print('len base features = ',len(features_names))
    print('example base features =',features_names[:5])
    
    ################### CV MODEL RANDOM SPLIT ###############

    gc.collect()
    
    for idx,model_type in enumerate(['xgboost']):

    
        X_train      = df_train[((df_train['Z_WEEK_DATE'] >= TRAIN_START) &(df_train['Z_WEEK_DATE'] < TEST_START))&
                               (df_train['Z_WEEK_DATE'] != '2022-04-11')].copy() 
        X_test       = df_train[(df_train['Z_WEEK_DATE'] >= TEST_START) & (df_train['Z_WEEK_DATE'] < TEST_END)].copy() 

        print('*'*20)
        print('MODEL = ',model_type)
        print('CV MODEL RANDOM SPLIT model_type = ',model_type,'c_model_v2')
        print('*'*20)

        y_train = X_train[TARGET]
        y_test =  X_test[TARGET]
        y_submission = df_test[TARGET]
        
        X_train = X_train[features_names]
        X_test  = X_test[features_names]
        X_submission = df_test[features_names]        
        
        X_train_Z_WEEK = X_train['Z_WEEK']
    
        del X_train['Z_WEEK']
        del X_test['Z_WEEK']
        del X_submission['Z_WEEK']

        print('X_train total cv',X_train.shape)
        print('X_test          ',X_test.shape)
        print('X_submission    ',X_submission.shape)

        model_version = 'c_model_v2'
        training.TARGET = TARGET
        training.SHIFT = SHIFT

        df_submission,df_feature_importance,metrics = training.training_model_cv(model_type,model_version,X_train,y_train,
                                                            X_test,y_test,X_submission,df_test,result,N_FOLDS,X_train_Z_WEEK)

        metrics.append('SEMANA_'+str(SHIFT))
        metrics.append('MODEL_BASE')
        metrics_dict_list.append(metrics)
    
        gc.collect()
        
        '''
        if idx ==0 :
            features_names = list(df_feature_importance['feature'][:30].values)
            
        ################### CV MODEL RANDOM SPLIT + FEATURE IMPORTANCE ###############
        important_features = list(df_feature_importance[:N_FEATURE_IMPORTANCE]['feature'])
        
        X_train      = df_train[(df_train['Z_WEEK_DATE'] >= TRAIN_START)&(df_train['Z_WEEK_DATE'] < TEST_START)].copy() 
        X_test       = df_train[(df_train['Z_WEEK_DATE'] >= TEST_START) & (df_train['Z_WEEK_DATE'] < TEST_END)].copy() 

        y_train = X_train[TARGET]
        y_test =  X_test[TARGET]
        y_submission = df_test[TARGET]

        X_train = X_train[important_features]
        X_test  = X_test[important_features]
        X_submission = df_test[important_features]

        print('X_train total cv',X_train.shape)
        print('X_test          ',X_test.shape)
        print('X_submission    ',X_submission.shape)

        print('*'*20)
        print('MODEL = ',model_type)
        print('CV MODEL RANDOM SPLIT + FEATURE IMPORTANCE model_type = ',model_type,'c_model_v3')
        print('*'*20)
        
        model_version = 'c_model_v3'
        training.TARGET = TARGET
        training.SHIFT = SHIFT


        df_submission,df_feature_importance,metrics = training.training_model_cv(model_type,model_version,X_train,y_train,
                                                               X_test,y_test,X_submission,df_test,result,N_FOLDS,X_train_Z_WEEK)
        
        metrics.append('SEMANA_'+str(SHIFT))
        metrics.append('MODEL_IMPORTANCE')
        metrics_dict_list.append(metrics)
        gc.collect()
        #'''
        pass

********************
********************
WEEK =  1
********************
********************
len base features =  250
example base features = ['shift_1_roll_4_std_Z_MODELO_Z_MARCA', 'enc_Z_PUNTO_VENTA_Z_DEPARTAMENTO_std', 'shift_2_roll_2_mean_Z_MODELO_Z_PUNTO_VENTA', 'enc_Z_MODELO_mean', 'shift_2_roll_2_mean_Z_PUNTO_VENTA_Z_MARCA']
********************
MODEL =  xgboost
CV MODEL RANDOM SPLIT model_type =  xgboost c_model_v2
********************
X_train total cv (1462363, 249)
X_test           (377384, 249)
X_submission     (471730, 249)
KFold(n_splits=3, random_state=100, shuffle=True)
********************  fold n°0
Training ...
directory_model =  ../../results/Demanda/xgboost/c_model_v2/fold_1/shift_1
create folder ../../results/Demanda/xgboost/c_model_v2/fold_1/shift_1
[08:15:40] WARNING: ../src/learner.cc:627: 
Parameters: { "scale_pos_weight", "silent", "verbose_eval" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  t

In [10]:
import pandas as pd

df = pd.DataFrame(metrics_dict_list,columns =['train rmse','val rmse','train mape','val mape','train mase','val mase','semana','model'])

In [11]:
df.head(10)

,train rmse,val rmse,train mape,val mape,train mase,val mase,semana,model
0,2.583225,2.155961,0.651793,0.699585,0.485844,0.502270,SEMANA_1,MODEL_BASE
1,2.990910,2.582859,0.675311,0.731605,0.541964,0.567690,SEMANA_2,MODEL_BASE
2,3.217231,3.019827,0.689128,0.767349,0.581907,0.621156,SEMANA_3,MODEL_BASE
3,3.281717,3.569227,0.695009,0.786081,0.606560,0.667025,SEMANA_4,MODEL_BASE


Final score mean_squared_errorweek = 51
Score week 51_rmse  : 1.129828390480792
Final score mean_absolute_percentage_errorweek = 51
Score week 51_mape  : 0.4859397529023634
Final score mean_absolute_errorweek = 51
Score  week 51_mase  : 0.20017442947773822
Final score mean_squared_errorweek = 52
Score week 52_rmse  : 1.3922219745356414
Final score mean_absolute_percentage_errorweek = 52
Score week 52_mape  : 3.0035044199836345
Final score mean_absolute_errorweek = 52
Score  week 52_mase  : 0.3604006065908547
Final score mean_squared_errorweek = 53
Score week 53_rmse  : 1.6076551720439693
Final score mean_absolute_percentage_errorweek = 53
Score week 53_mape  : 1.2471906195221085
Final score mean_absolute_errorweek = 53
Score  week 53_mase  : 0.26027708620955764
Final score mean_squared_errorweek = 54
Score week 54_rmse  : 1.7885096355912606
Final score mean_absolute_percentage_errorweek = 54
Score week 54_mape  : 0.9445388431460574
Final score mean_absolute_errorweek = 54
Score  week 54_mase  : 0.40160796886468336
    
    
Final score mean_squared_errorweek = 51
Score week 51_rmse  : 1.0927411093690542
Final score mean_absolute_percentage_errorweek = 51
Score week 51_mape  : 1.063006267005036
Final score mean_absolute_errorweek = 51
Score  week 51_mase  : 0.20562774085672958
Final score mean_squared_errorweek = 52
Score week 52_rmse  : 1.4804781189435823
Final score mean_absolute_percentage_errorweek = 52
Score week 52_mape  : 2.965989977459292
Final score mean_absolute_errorweek = 52
Score  week 52_mase  : 0.3460008271666122
Final score mean_squared_errorweek = 53
Score week 53_rmse  : 1.7157894190144294
Final score mean_absolute_percentage_errorweek = 53
Score week 53_mape  : 1.742155191722578
Final score mean_absolute_errorweek = 53
Score  week 53_mase  : 0.26688131125992776
Final score mean_squared_errorweek = 54
Score week 54_rmse  : 2.0156692663053084
Final score mean_absolute_percentage_errorweek = 54
Score week 54_mape  : 0.9175125868517022
Final score mean_absolute_errorweek = 54
Score  week 54_mase  : 0.40538876925771883
Final score mean_squared_errorweek = 55